# Pump It Up Competition

**Loading dataset and exploration**

In [12]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
feature=pd.read_csv("E:\\PumpItUp\\training_values.csv", index_col=0)
label=pd.read_csv("E:\\PumpItUp\\training_labels.csv", index_col=0)
test=pd.read_csv("E:\\PumpItUp\\test_values.csv", index_col=0)

**Sumbission Function**

In [3]:
def submission(X_test, model):
    y_pred=model.predict(X_test)
    result=pd.Series(data=y_pred,
                index=X_test.index,
                name='status_group')
    date=pd.Timestamp.now().strftime(format="%m_%d_%Y_%H-%M_")
    result.to_csv(f"E:\\PumpItUp\\{date}result.csv", index=True, header=True)

**Predictive Model Pipeline**

In [4]:
X_train, X_val, y_train, y_val=train_test_split(feature, label, test_size=0.2, random_state=42)

In [5]:
num_feat=X_train.select_dtypes(include='number').columns.to_list()
cat_feat=X_train.select_dtypes(exclude='number').columns.to_list()

In [14]:
num_pipe=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipe=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])

ct_all=ColumnTransformer(remainder='drop',
                    transformers=[
                        ('numeric',num_pipe,num_feat),
                        ('categorical',cat_pipe,cat_feat)
                    ])


rfc_params = {'n_estimators':[2,5,10,20,50,75,150],
              'criterion':['gini', 'entropy'],
              'max_depth':[2,5,10,20,50,None],
              'min_samples_split':[2,5,10,20]}


grid_rfc = GridSearchCV(RandomForestClassifier(), 
                        rfc_params, 
                        cv=5, 
                        scoring='accuracy')

model_2=Pipeline([
    ('transformer',ct_all),
    ('predictor', grid_rfc)
])

In [15]:
model_2.fit(X_train, y_train);

C:\Users\Ferdous\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ferdous\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ferdous\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Ferdous\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:516: DataConversionWarning: A column-vec

KeyboardInterrupt: 

In [ ]:
y_pred_2=model_2.predict(X_val)

In [ ]:
#model_2 score
print('In sample accuracy: ',model_2.score(X_train, y_train))
print('Out sample accuracy: ',accuracy_score(y_pred_2, y_val))

In [10]:
#model_2.fit(feature,label)

In [ ]:
#Final submission
submission(test, model_2)